In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
from urllib import parse
import urllib.request as req
import time
import pandas as pd


import math

In [2]:
# Chrome Browser 와 Chrome Driver Version 확인하기
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.get('https://paikdabang.com/store/')


<!-- 시도 클릭-> i 클릭 -> 구군 클릭 -> j 클릭 -> next버튼 체크, 있으면 계속  -->
전체-전체로 검색, 총 1260개의 매장 확인, 10으로 나눠서 126번

In [3]:
#max_count
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')
maxCount = soup.select_one('#max_count')
maxCount = maxCount.string

maxCount = int(maxCount)
maxPage = math.ceil(maxCount/10)
print(maxPage)


126


In [4]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')
shopName = []
shopAddress=[]
shopArea=[]
errorList =[]
count = 2
shops = soup.select('.tr_list')
# shops

for i in range(0,maxPage):
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    shops = soup.select('.tr_list')
    for shop in shops:
        shopName.append(shop.select_one('.td_shop').text)
        shopArea.append(shop.select_one('.td_area').text)
        shopAddress.append(shop.select_one('.td_addr').text.replace(",",""))
    try: driver.find_element(By.XPATH, '//*[@id="pagination"]/li['+ str(count) +']').click()
    except: errorList.append(f'Error number : {str(i)} 매장명 :{shop.select_one(".td_shop").text}')
    else:
        time.sleep(1)
        if count < 6:
            count += 2
        else:
            count = 6
    


In [5]:
errorList

[]

In [7]:
len(shopName)

1260

---
# Save the CSV

In [19]:
df = pd.DataFrame(data=shopArea, columns=['Area']) #< columns는 [] 로 작성해야댐 
df['ShopName'] = pd.Series(shopName)
df['Address'] = pd.Series(shopAddress)

/var/folders/j9/zr30dj9x51x0cxznxz0064t80000gn/T/ipykernel_73792/3210349423.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['ShopName'] = pd.Series(shopName)
/var/folders/j9/zr30dj9x51x0cxznxz0064t80000gn/T/ipykernel_73792/3210349423.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['Address'] = pd.Series(shopAddress)


In [15]:
df.to_csv('../Data/PaiksCoffee.csv', encoding='utf-8', index=False)